In [ ]:
def cluster_fires(gdf, eps=0.01, min_samples=1):
    """
    Given a GeoDataFrame of fire points, create spatial clusters
    :param gdf: GeoDataFrame of fire points
    :param eps: The maximum distance between two samples for one to be considered as in the neighborhood of the other
    :param min_samples: The number of samples in a neighborhood for a point to be considered as a core point
    :return: GeoDataFrame of fire points with an additional column 'label' indicating the cluster each point belongs to
    """

    # Perform DBSCAN clustering
    coords = gdf[['longitude', 'latitude']].values
    db = DBSCAN(eps=eps, min_samples=min_samples).fit(coords)

    # Add cluster labels to the dataframe
    gdf['label'] = db.labels_

    return gdf

def filter_clusters(gdf, min_cluster_size=10, min_high_confidence=1):
    """
    Filter out clusters that have fewer points, and fewer high confidence points, than the two thresholds
    :param gdf: GeoDataFrame of fire points with 'label' column indicating the cluster each point belongs to
    :param min_cluster_size: Minimum number of points in a cluster for it to be kept
    :param min_high_confidence: Minimum number of high confidence points in a cluster for it to be kept
    :return: GeoDataFrame of fire points in clusters that meet both thresholds
    """

    # Count the number of points in each cluster
    cluster_counts = gdf['label'].value_counts()

    # Count the number of high confidence points in each cluster
    high_confidence_counts = gdf.loc[gdf['confidence'] == 'h']['label'].value_counts()

    # Filter out small clusters and clusters with too few high confidence points
    valid_clusters = cluster_counts[(cluster_counts >= min_cluster_size) & (high_confidence_counts >= min_high_confidence)].index
    gdf = gdf[gdf['label'].isin(valid_clusters)]

    return gdf